In [3]:
# 1 ===========================================================
import sys
sys.path.append("C:/Users/danie/Documents/School/MEEN537/HW")
sys.path.append("/home/daniel/Documents/MEEN537/HW/")
import kinematics as kin
# from visualization import VizScene
import sympy as sp
import numpy as np
import time
np.set_printoptions(precision=4, suppress=True)

# Define the arm
dh = [[0, 0, 0, np.pi/2.0],
      [0, 0, 0.4318, 0],
      [0, 0.15, 0.02, -np.pi/2],
      [0, 0.4318, 0, np.pi/2],
      [0, 0, 0, -np.pi/2],
      [0, 0.4, 0, 0]]

jt_types = ['r']*6
T_tool_in_6 = np.array([[0,  0, 1, 0], 
                        [0,  1, 0, 0], 
                        [-1, 0, 0, 0.2], 
                        [0,  0, 0, 1]])

arm = kin.SerialArm(dh, jt=jt_types, tip=T_tool_in_6)
q0 = [0]*6

# using FK to get rotation from 0 to 6 
T_6_in_0 = arm.fk(q0)

# finding rotation to describe transfrom to take Jacobian in frame zero, to the tool frame 
# (indices should cancel to give J a "tool" superscript)
R_0_in_6 = T_6_in_0[0:3,0:3].T
R_6_in_tool = T_tool_in_6[0:3,0:3].T

# finding the vector from frame 6 to tool, buit in the tool frame
p_from_6_to_tool_in_frame_6 = T_tool_in_6[0:3,3]

Z_0_in_6_from_6_to_6 = arm.Z_shift(R=R_0_in_6)
Z_6_in_tool_from_6_to_tool = arm.Z_shift(R=R_6_in_tool, p=p_from_6_to_tool_in_frame_6, p_frame='i')

J_at_6_in_frame_0 = arm.jacob(q0)

# shifting the Jacobian, in two steps because it's easier this way.
J_at_tool_in_tool_frame = Z_6_in_tool_from_6_to_tool @ Z_0_in_6_from_6_to_6 @ J_at_6_in_frame_0



print(J_at_tool_in_tool_frame)

# See how the velocities compare for J in tool at tool and J in base frame at end effector
# qdot = np.array([1,0,0,0,0,0])

# print(J_06 @ qdot)
# print()
# print(J_tooltool @ qdot)


[[ 0.     -0.4518 -0.02    0.      0.      0.    ]
 [ 0.4518  0.      0.      0.      0.      0.    ]
 [ 0.15   -1.0318 -1.0318  0.     -0.6     0.    ]
 [-1.     -0.     -0.     -1.     -0.     -1.    ]
 [ 0.     -1.     -1.      0.     -1.      0.    ]
 [ 0.      0.      0.      0.      0.      0.    ]]


In [ ]:
import sys
sys.path.append("C:/Users/danie/Documents/School/MEEN537/HW")
sys.path.append("/home/daniel/Documents/MEEN537/HW/")
import kinematics as kin
# from visualization import VizScene
import sympy as sp
import numpy as np
import dynamics as dyn

dh = [[0, 0, 0.4, 0],
      [0, 0, 0.4, 0],
      [0, 0, 0.4, 0]]

n = 3
jt_types = ['r'] * n
link_mass = [1] * n   # kg per link
r_coms = [np.array([-0.2, 0, 0])] * n
Izz = 0.1

link_inertias = []
for i in range(n):
      link_inertias.append(np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0.01]]))

arm = dyn.SerialArmDyn(dh,
                        jt=jt_types,
                        mass=link_mass,
                        r_com=r_coms,
                        link_inertia=link_inertias)

q = [np.pi/4.0]*n
qd = [np.pi/6, -np.pi/4, np.pi/3]
qdd = [-np.pi/6, np.pi/3, np.pi/6]
arm.rne(q, qd, qdd)